In [2]:
import pandas as pd 
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import pickle
import lxml

In [5]:
# get this data 
pd.read_parquet("[2022, 2023, 2024]_error.parquet.gzip")

,season,month,error
0,2023,november,invalid season link
1,2023,january,invalid season link
2,2023,march,invalid season link
3,2024,april,unable to scrape table


In [10]:
pd.read_parquet("[2022, 2023, 2024]_game_stats.parquet.gzip").to_csv("[2022, 2023, 2024]_game_stats.csv")

In [51]:
def get_season_schedule(filename, seasons = [2015]):
    '''
    Gets the season schedule (games played throughout the season) up until the playoffs.
    creates a dataframe to inform us of any potential errors and allows code to not break because of errors
    '''
    season_schedule_table = pd.DataFrame(columns = ["season", "date", "away_team_id", "home_team_id", "arena", "link"])
    # months = ["november", "january", "march"]
    months = ["april"]
    # ["october", "november", "december", "january", "february", "march", "april"]
    
    for year in seasons:
        print(f"getting {year} data....")
        for month in months:
            month_schedule = f"https://www.basketball-reference.com/leagues/NBA_{year}_games-{month}.html"
            print(month_schedule)
            time.sleep(random.randint(2, 6))
            #checks whether the link is valid and lets us know if any connections didnt work by saving to erros df. some errors here are good!
            try:
                month_schedule_link = urlopen(month_schedule)
                soup = BeautifulSoup(month_schedule_link, features="lxml")
            except:
                #the month may not have been a part of the season/year so each year will have some error months. this is really here for the covid seasons and high level error checking
                errors = pd.DataFrame({"season": [year], "month":[month], "error": ["invalid season link"]})
                try:
                    name_of_file = f"{filename}_error.csv"
                    errors = pd.concat([pd.read_parquet(name_of_file), errors], ignore_index = True)
                    errors.to_parquet(name_of_file)
                except:
                    name_of_file = f"{filename}_error.csv"
                    errors.to_parquet(name_of_file)
                continue
            try:
                schedule_table = soup.find('table', attrs={'id':'schedule'})
                games = schedule_table.tbody.findAll("tr")
                season = []
                dates = []
                away_team_ids = []
                home_team_ids = []
                arenas = []
                links = []
                for game in games:
                #avoinding playoffs schedule
                    if game.th.string == "Playoffs":
                        break
                    season.append(year)
                    other_stats = game.findAll("td")
                    date = game.th.a.string
                    dates.append(date)
                    away_team = other_stats[1].a.get('href')[7:10]
                    away_team_ids.append(away_team)
                    home_team = other_stats[3].a.get('href')[7:10]
                    home_team_ids.append(home_team)
                    boxscore = other_stats[5].a.get('href')
                    links.append(boxscore)
                    arena = other_stats[8].string
                    arenas.append(arena)
                season_month = pd.DataFrame({"season": season,
                                             "date": dates,
                                             "away_team_id": away_team_ids,
                                             "home_team_id": home_team_ids,
                                              "arena":arenas,
                                             "link":links
                                            })
            except:
                #table error most likely 2020 april, where games werent played
                errors = pd.DataFrame({"season": [year], "month":[month], "error": ["unable to scrape table"]})
                try:
                    name_of_file = f"{filename}_error.csv"
                    errors = pd.concat([pd.read_parquet(name_of_file), errors], ignore_index = True)
                    errors.to_parquet(name_of_file)
                except:
                    name_of_file = f"{filename}_error.csv"
                    errors.to_parquet(name_of_file)
                continue
            # try to append to an existing file, if not exist create it
            try:
                name_of_file = f"{filename}"
                season_games = pd.concat([pd.read_csv(name_of_file), season_month], ignore_index = True)
                season_games.to_csv(name_of_file)
            except:
                name_of_file = f"{filename}"
                season_month.to_csv(name_of_file)
            print(month, "completed")
    return pd.read_parquet(name_of_file)

In [52]:
def get_game_stats(dataframe, filename):
    """
    gets individual game statistics given a seasons games as a pandas df
    """
    count = 1
    flag = 0

    game_stats = pd.DataFrame(columns = ["date",
            "home_team",
            "home_fg",
            "home_fga",
            "home_fg_pct",
            "home_fg3",
            "home_fg3a",
            "home_fg3_pct",
            "home_ft",
            "home_fta",
            "home_ft_pct",
            "home_orb",
            "home_drb",
            "home_trb",
            "home_ast",
            "home_stl",
            "home_blk",
            "home_tov",
            "home_pf",
            "home_pts",
            "away_team",
            "away_fg",
            "away_fga",
            "away_fg_pct",
            "away_fg3",
            "away_fg3a",
            "away_fg3_pct",
            "away_ft",
            "away_fta",
            "away_ft_pct",
            "away_orb",
            "away_drb",
            "away_trb",
            "away_ast",
            "away_stl",
            "away_blk",
            "away_tov",
            "away_pf",
            "away_pts",
            "arena"])
    
    print("getitng individual game stats", end = " ")

    for game in dataframe.iterrows():
        print('.', end="")
        game = game[1]
        home_team_id = game["home_team_id"]
        away_team_id = game["away_team_id"]
        arena = game["arena"]
        ext = game["link"]
        this_season = game["season"]
        try:
            box_score_url = f"https://www.basketball-reference.com{ext}"
            time.sleep(random.randint(5, 10))
            open_link = urlopen(box_score_url)
            soup = BeautifulSoup(open_link, features="lxml")
        except Exception as e:
            print(e)
            error = pd.DataFrame({"season": game["season"],
            "date": game["date"],
            "away_team_id": away_team_id,
            "home_team_id": home_team_id,
            "arena": arena,
            "link": ext,
            "error": ["404: invalid game link"]})
            try:
                name_of_file = f"{filename}_error.csv"
                errors = pd.concat([pd.read_parquet(name_of_file), error], ignore_index = True)
                errors.to_csv(name_of_file)
            except:
                name_of_file = f"{filename}_error.csv"
                error.to_csv(name_of_file)
            continue
        try:
            home_stats = soup.find('table', attrs={'id':f"box-{home_team_id}-game-basic"}).tfoot.tr
            home_fg = home_stats.find('td', attrs = {'data-stat': 'fg'}).string
            home_fga = home_stats.find('td', attrs = {'data-stat': 'fga'}).string
            home_fg_pct = home_stats.find('td', attrs = {'data-stat': 'fg_pct'}).string
            home_fg3 = home_stats.find('td', attrs = {'data-stat': 'fg3'}).string
            home_fg3a = home_stats.find('td', attrs = {'data-stat': 'fg3a'}).string
            home_fg3_pct = home_stats.find('td', attrs = {'data-stat': 'fg3_pct'}).string
            home_ft = home_stats.find('td', attrs = {'data-stat': 'ft'}).string
            home_fta = home_stats.find('td', attrs = {'data-stat': 'fta'}).string
            home_ft_pct = home_stats.find('td', attrs = {'data-stat': 'ft_pct'}).string
            home_orb = home_stats.find('td', attrs = {'data-stat': 'orb'}).string
            home_drb = home_stats.find('td', attrs = {'data-stat': 'drb'}).string
            home_trb = home_stats.find('td', attrs = {'data-stat': 'trb'}).string
            home_ast = home_stats.find('td', attrs = {'data-stat': 'ast'}).string
            home_stl = home_stats.find('td', attrs = {'data-stat': 'stl'}).string
            home_blk = home_stats.find('td', attrs = {'data-stat': 'blk'}).string
            home_tov = home_stats.find('td', attrs = {'data-stat': 'tov'}).string
            home_pf = home_stats.find('td', attrs = {'data-stat': 'pf'}).string
            home_pts = home_stats.find('td', attrs = {'data-stat': 'pts'}).string

            away_stats = soup.find('table', attrs={'id':f"box-{away_team_id}-game-basic"}).tfoot.tr
            away_fg = away_stats.find('td', attrs = {'data-stat': 'fg'}).string
            away_fga = away_stats.find('td', attrs = {'data-stat': 'fga'}).string
            away_fg_pct = away_stats.find('td', attrs = {'data-stat': 'fg_pct'}).string
            away_fg3 = away_stats.find('td', attrs = {'data-stat': 'fg3'}).string
            away_fg3a = away_stats.find('td', attrs = {'data-stat': 'fg3a'}).string
            away_fg3_pct = away_stats.find('td', attrs = {'data-stat': 'fg3_pct'}).string
            away_ft = away_stats.find('td', attrs = {'data-stat': 'ft'}).string
            away_fta = away_stats.find('td', attrs = {'data-stat': 'fta'}).string
            away_ft_pct = away_stats.find('td', attrs = {'data-stat': 'ft_pct'}).string
            away_orb = away_stats.find('td', attrs = {'data-stat': 'orb'}).string
            away_drb = away_stats.find('td', attrs = {'data-stat': 'drb'}).string
            away_trb = away_stats.find('td', attrs = {'data-stat': 'trb'}).string
            away_ast = away_stats.find('td', attrs = {'data-stat': 'ast'}).string
            away_stl = away_stats.find('td', attrs = {'data-stat': 'stl'}).string
            away_blk = away_stats.find('td', attrs = {'data-stat': 'blk'}).string
            away_tov = away_stats.find('td', attrs = {'data-stat': 'tov'}).string
            away_pf = away_stats.find('td', attrs = {'data-stat': 'pf'}).string
            away_pts = away_stats.find('td', attrs = {'data-stat': 'pts'}).string

            this_game_stats = pd.DataFrame({
            "date": [game["date"]],
            "home_team":[home_team_id],
            "home_fg":[home_fg],
            "home_fga":[home_fga],
            "home_fg_pct":[home_fg_pct],
            "home_fg3":[home_fg3],
            "home_fg3a":[home_fg3a],
            "home_fg3_pct":[home_fg3_pct],
            "home_ft":[home_ft],
            "home_fta":[home_fta],
            "home_ft_pct":[home_ft_pct],
            "home_orb":[home_orb],
            "home_drb":[home_drb],
            "home_trb":[home_trb],
            "home_ast":[home_ast],
            "home_stl":[home_stl],
            "home_blk":[home_blk],
            "home_tov":[home_tov],
            "home_pf":[home_pf],
            "home_pts":[home_pts],
            "away_team":[away_team_id],
            "away_fg":[away_fg],
            "away_fga":[away_fga],
            "away_fg_pct":[away_fg_pct],
            "away_fg3":[away_fg3],
            "away_fg3a":[away_fg3a],
            "away_fg3_pct":[away_fg3_pct],
            "away_ft":[away_ft],
            "away_fta":[away_fta],
            "away_ft_pct":[away_ft_pct],
            "away_orb":[away_orb],
            "away_drb":[away_drb],
            "away_trb":[away_trb],
            "away_ast":[away_ast],
            "away_stl":[away_stl],
            "away_blk":[away_blk],
            "away_tov":[away_tov],
            "away_pf":[away_pf],
            "away_pts":[away_pts],
            "arena": [arena]})
            
            game_stats = pd.concat([game_stats, this_game_stats], ignore_index = True)
            
        except Exception as e:
            print(e)
            error = pd.DataFrame({"season": game["season"],
            "date": game["date"],
            "away_team_id": away_team_id,
            "home_team_id": home_team_id,
            "arena": arena,
            "link": ext,
            "error": ["error scraping table"]})
            flag = 1
            try:
                name_of_file = f"{filename}_error.csv"
                errors = pd.concat([pd.read_parquet(name_of_file), error], ignore_index = True)
                errors.to_csv(name_of_file)
            except Exception as e:
                print(e)
                name_of_file = f"{filename}_error.csv"
                error.to_csv(name_of_file)
            continue


    if count % 100 == 0:
        name_of_file = f"{filename}.csv"
        try:
            existing_df = pd.read_csv(name_of_file)
            combined_df = pd.concat([game_stats, existing_df], ignore_index = True)
            combined_df.to_csv(name_of_file)
        except Exception as e:
            print(e)
            game_stats.to_csv(name_of_file)
       
        scraped = len(pd.read_csv(name_of_file))
        print(f"\nstats scraped from {scraped} of {len(dataframe)} in {this_season}")
        game_stats = pd.DataFrame(columns = game_stats.columns)

        
    if count == len(dataframe):
        name_of_file = f"{filename}.csv"
        existing_df = pd.read_csv(name_of_file)
        combined_df = pd.concat([game_stats, existing_df], ignore_index = True)
        combined_df.to_csv(name_of_file)
    count+=1

    if flag == 1:
        print("errors: ", len(pd.read_csv(f"{filename}_error.csv")))
    return game_stats

In [58]:
get_season_schedule("[2024]", [2024])

getting 2024 data....
https://www.basketball-reference.com/leagues/NBA_2024_games-april.html


,season,month,error
0,2024,april,unable to scrape table


In [54]:
schedule_2023 = pd.read_parquet("[2023].parquet.gzip")
get_game_stats(schedule_2023, "missing_2023_gs")

getitng individual game stats .................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

,date,home_team,home_fg,home_fga,home_fg_pct,home_fg3,home_fg3a,home_fg3_pct,home_ft,home_fta,...,away_orb,away_drb,away_trb,away_ast,away_stl,away_blk,away_tov,away_pf,away_pts,arena
0,"Tue, Nov 1, 2022",BRK,36,74,.486,12,32,.375,15,19,...,11,33,44,20,8,4,9,13,108,Barclays Center
1,"Tue, Nov 1, 2022",MIA,40,88,.455,16,42,.381,20,20,...,10,31,41,31,8,5,20,22,109,Kaseya Center
2,"Tue, Nov 1, 2022",OKC,45,95,.474,10,34,.294,16,20,...,12,37,49,20,7,10,24,14,108,Paycom Center
3,"Tue, Nov 1, 2022",PHO,41,89,.461,17,43,.395,17,22,...,10,32,42,24,5,4,16,24,107,Footprint Center
4,"Wed, Nov 2, 2022",PHI,39,84,.464,19,36,.528,14,20,...,7,36,43,27,4,9,7,18,121,Wells Fargo Center
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
668,"Fri, Mar 31, 2023",MEM,43,96,.448,9,33,.273,13,17,...,12,38,50,26,5,8,17,14,94,FedEx Forum
669,"Fri, Mar 31, 2023",MIN,41,95,.432,13,39,.333,16,19,...,15,38,53,28,10,5,16,17,123,Target Center
670,"Fri, Mar 31, 2023",GSW,47,93,.505,21,49,.429,15,21,...,14,32,46,27,8,4,13,20,115,Chase Center
671,"Fri, Mar 31, 2023",POR,40,85,.471,19,38,.500,15,23,...,8,33,41,38,11,7,13,21,138,Moda Center


In [57]:
scraped_game_stats = _54
scraped_game_stats.to_csv("[2022, 2023]_game_stats.csv")